# Sprachübergreifende Textalignierung

Blockseminar Studiengang "Digitale Methodik in den Geistes- und Kulturwissenschaften" (18.1.2020, 8.2.2020, 15.2.2020)




In [13]:
import os

in_dir = "./data/constitutions/"

# we create a dictionary with our constitutions:
sources = {}

for file in sorted(os.listdir(in_dir)):
    key = os.path.basename(file).split(os.extsep)[0]
    with open(in_dir + '/' + file) as f:
        sources[key] = f.read()

# and a list of available constitutions for quick lookup:
constitutions = list(sources.keys())

print ("{} files read:".format(len(constitutions)))
print (constitutions)

5 files read:
['1867_-_GB-NA_-_Great_Britain_-_british_north_american_act', '1909_-_GB-SA_-_Great_Britain_-_british_south_africa_act', '1935_-_GB-BU_-_Great_Britain_-_british_government_of_burma_act', '1937_-_IE_-_Ireland_-_constitution_of_ireland', '1948_-_BU_-_Burma_-_constitution_of_burma']


In [14]:
from nltk import tokenize
# nltk.download('punkt')

sentences = {}
nos = {}
for c in constitutions:
    t = tokenize.sent_tokenize(sources[c])
    nos[c] = len(t)
    for i, s in enumerate(t):
        sentences[c + '_' + str(i)] = s

boundary = len(sentences) - nos['1948_-_BU_-_Burma_-_constitution_of_burma']
print("Corpus has {} sentences.".format(len(sentences)))
print("1948_-_BU_-_Burma_-_constitution_of_burma has {}.\n".format(nos['1948_-_BU_-_Burma_-_constitution_of_burma']))

print("Its first 3 sentences are:\n{}".format([sentences['1948_-_BU_-_Burma_-_constitution_of_burma_0'],\
                                              sentences['1948_-_BU_-_Burma_-_constitution_of_burma_1'],\
                                              sentences['1948_-_BU_-_Burma_-_constitution_of_burma_2']]))

Corpus has 3240 sentences.
1948_-_BU_-_Burma_-_constitution_of_burma has 963.

Its first 3 sentences are:
['The Constitution of the Union of Burma, 24 September 1947, Effective 4 January 1948\n\nPREAMBLE\nWE, THE PEOPLE OF BURMA including the Frontier Areas and the Karenni States, Determined to establish in strength and unity a\nSOVEREIGN INDEPENDENT STATE, To maintain social order on the basis of the eternal principles of JUSTICE, LIBERTY AND EQUALITY\nand To guarantee and secure to all citizens JUSTICE social, economic and political; LIBERTY of thought, expression, belief,\nfaith, worship, vocation, association and action; EQUALITY of status, of opportunity and before the law, IN OUR CONSTITUENT\nASSEMBLY this Tenth day of Thadingyut waxing, 1309 B.E.', '(Twentyfourth day of September, 1947 A.D.), DO HEREBY ADOPT,\nENACT AND GIVE TO OURSELVES THIS CONSTITUTION.', 'CHAPTER I.']


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word', strip_accents='unicode', stop_words=["the", "of", "and"])
dfm = vectorizer.fit_transform(sentences.values())

print(dfm.shape)
print(type(dfm))
print(dfm.toarray())

(3240, 4685)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

target = dfm[boundary:]
sources = dfm[:boundary,]
print(target.shape)
print(sources.shape)

simils = cosine_similarity(target, sources)
print(simils.shape)

(963, 4685)
(2277, 4685)
(963, 2277)


In [17]:
import numpy as np

np.argmax(simils, axis=1)

array([1703, 1898,   89,  589,    0, 1741,    0, 1334,    0,  499,    0,
       1905,    0, 1757,    0, 1765,    0, 1765,    0, 2099,    6, 2199,
        589,    0,  536, 1911,  992,  927,  994, 1475,  997,  256, 2199,
        999, 1313,  529, 1680, 1005, 1749, 2199, 1007, 2206, 1009, 2213,
       2216, 2218, 2198, 1653, 1012, 2161, 1014,  948,    0, 1571, 1011,
       1016, 2241,    0, 2155,    0, 2200, 1018, 2239, 2240, 2243, 2272,
       2199, 1020, 2014, 2199, 1022, 2233, 2234, 1888, 2215, 2272, 1297,
       1024, 1426, 1349, 1026, 2272, 1653, 2102, 1198, 1043, 1200, 2102,
       1206, 1230, 1209, 2274,   20, 1659, 1215, 1902, 1690,  615, 1947,
       1217, 1916, 1349,   38, 2247, 1220, 2247, 2248, 1226, 2234,  797,
       1232, 2230, 1236, 2249, 1240, 1995, 1243, 2258, 2224, 1247, 2130,
       1251, 1577, 1254,  589, 2201, 1258,    7, 1262, 2200, 1376,  645,
        531,  648,  789,  789,  570, 1283, 1919, 1286, 1920, 1931, 1294,
       1932, 1933, 1934,  659, 1923, 1833, 1303, 19